In [1]:
!pip install transformers
!pip install datasets
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.4 MB/s eta 0:00:

In [2]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.9 MB/s eta 0:00:00


In [13]:
import torch
import numpy as np
import pandas as pd
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, EarlyStoppingCallback)
from transformers import BartTokenizer, BartForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset, load_metric
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig
from transformers import AutoTokenizer, LongT5ForConditionalGeneration
from transformers import LEDTokenizer, LEDForConditionalGeneration

In [4]:
train_import = pd.read_json('/content/drive/MyDrive/NLP Project Files/Maddy/data/train.jsonl', lines = True)
test_import = pd.read_json('/content/drive/MyDrive/NLP Project Files/Maddy/data/validation.jsonl', lines = True)

train_import['tags'] = train_import['tags'].map(lambda x: x[0])
test_import['tags'] = test_import['tags'].map(lambda x: x[0])

train_import = train_import[train_import['tags'] == 'phrase']
test_import = test_import[test_import['tags'] == 'phrase']

train_import['spoiler'] = train_import['spoiler'].map(lambda x: x[0])
test_import['spoiler'] = test_import['spoiler'].map(lambda x: x[0])

<ipython-input-4-6cfabc379e98>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_import['spoiler'] = train_import['spoiler'].map(lambda x: x[0])


In [7]:
train_import['processedParagpraphs'] = train_import['targetParagraphs'].map(lambda x: '. '.join(x))
test_import['processedParagpraphs'] = test_import['targetParagraphs'].map(lambda x: '. '.join(x))

train_import['processedText'] = train_import['postText'].map(lambda x: x[0])
test_import['processedText'] = test_import['postText'].map(lambda x: x[0])

In [5]:
def preprocess_data(example):
    input_text = "condition: " + example['processedText'] + " paragraph: " + example['processedParagpraphs']
    target_text = example['spoiler']
    return {"input_text": input_text, "target_text": target_text}

In [8]:
train_dataset = Dataset.from_pandas(train_import)
test_dataset = Dataset.from_pandas(test_import)

train_dataset_processed = train_dataset.map(preprocess_data)
test_dataset_processed = test_dataset.map(preprocess_data)

Map:   0%|          | 0/1367 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

In [14]:
model_name = "allenai/led-base-16384"

tokenizer = LEDTokenizer.from_pretrained(model_name)
model = LEDForConditionalGeneration.from_pretrained(model_name)

In [18]:
bleu_metric = load_metric("bleu")
meteor_metric = load_metric("meteor")
bertscore_metric = load_metric("bertscore")

<ipython-input-18-6ad6bd8f57b9>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("bleu")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [15]:
#out_1 = []
#out_2 = []

def compute_metrics(eval_pred):
    logits_full, labels = eval_pred

    #out_1.append(logits_full[0])
    #out_2.append(logits_full[1])

    predicted_token_ids = torch.argmax(torch.tensor(logits_full[0]), dim=-1).tolist()
    decoded_preds = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds_bleu = [pred.split() for pred in decoded_preds]
    decoded_labels_bleu = [[pred.split()] for pred in decoded_labels]
    
    bleu_score = bleu_metric.compute(predictions=decoded_preds_bleu, references= decoded_labels_bleu)
    meteor_score = meteor_metric.compute(predictions=decoded_preds, references=decoded_labels)
    bertscore_score = bertscore_metric.compute(predictions=decoded_preds, references=decoded_labels, lang = 'en')
    
    return {"bleu": bleu_score["bleu"], "meteor": meteor_score["meteor"], "bertscore": np.mean(bertscore_score["f1"])}

In [16]:
def encode_data(batch):
    input_ids = tokenizer(batch["input_text"], padding="max_length", truncation=True, max_length= 2040, return_tensors="pt").input_ids
    target_ids = tokenizer(batch["target_text"], padding="max_length", truncation=True, max_length= 11, return_tensors="pt").input_ids
    return {"input_ids": input_ids, "labels": target_ids}

encoded_train_dataset = train_dataset_processed.map(encode_data, batched=True)
encoded_test_dataset = test_dataset_processed.map(encode_data, batched=True)

columns_to_remove = ['uuid', 'postId', 'postText', 'postPlatform', 'targetParagraphs', 'targetTitle', 'targetDescription', 'targetKeywords', 'targetMedia', 'targetUrl', 'provenance', 'spoiler', 'spoilerPositions', 'tags', 'processedParagpraphs', 'processedText', '__index_level_0__']
encoded_train_dataset = encoded_train_dataset.remove_columns(columns_to_remove)
encoded_test_dataset = encoded_test_dataset.remove_columns(columns_to_remove)


Map:   0%|          | 0/1367 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

In [ ]:
import shutil

directory_path = f"/content/drive/MyDrive/NLP Project Files/conditional_generation/phrase/results/google"
shutil.rmtree(directory_path)

In [19]:
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/NLP Project Files/conditional_generation/phrase/results/{model_name}_results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    logging_steps=100,
    save_steps=0,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch"

)

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience= 3)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

metrics = trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Meteor,Bertscore
1,0.200900,0.509900,0.456959,0.556509,0.958826
2,0.167100,0.519457,0.514048,0.576907,0.962028
3,0.087500,0.548781,0.490654,0.555960,0.960426
4,0.036000,0.602405,0.503875,0.554115,0.961858
5,0.016800,0.585455,0.551866,0.570700,0.963532
6,0.008400,0.603959,0.539516,0.576462,0.964210
7,0.003700,0.608822,0.553717,0.576437,0.962401
8,0.001600,0.627002,0.534867,0.570840,0.963612
9,0.000600,0.630640,0.549505,0.578276,0.963149
10,0.000600,0.631495,0.550248,0.577996,0.963070


In [20]:
metrics 

{'eval_loss': 0.6088220477104187,
 'eval_bleu': 0.5537172308351904,
 'eval_meteor': 0.576437022359746,
 'eval_bertscore': 0.9624014092915094,
 'eval_runtime': 14.138,
 'eval_samples_per_second': 23.695,
 'eval_steps_per_second': 5.941,
 'epoch': 10.0}

In [21]:
best_model_checkpoint = trainer.state.best_model_checkpoint
print(f"Best model checkpoint: {best_model_checkpoint}")


Best model checkpoint: /content/drive/MyDrive/NLP Project Files/conditional_generation/phrase/results/allenai/led-base-16384_results/checkpoint-2394
